

# Pemrosesan Paralel dengan Dask
* **Produk yang digunakan:**
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)
* **Prasyarat**: Pengguna notebook ini harus memiliki pemahaman dasar tentang:
  * Cara menjalankan [Jupyter notebook](http://43.218.254.133:8888/notebooks/panduan-pengguna/01_Jupyter_notebooks.ipynb)
  * Memeriksa produk dan pengukuran yang tersedia di [Piksel Products and measurement](http://43.218.254.133:8888/notebooks/panduan-pengguna/02_Product_dan_measurement.ipynb)
  * Cara [membuka data Piksel](http://43.218.254.133:8888/notebooks/panduan-pengguna/03_Membuka_data.ipynb)
  * Cara [plotting data](http://43.218.254.133:8888/notebooks/panduan-pengguna/04_Plotting.ipynb)
  * Cara menjalankan [analisis dasar](http://43.218.254.133:8888/notebooks/panduan-pengguna/05_Analisis_dasar.ipynb)




## Latar Belakang
[Dask](https://dask.org/) adalah alat yang berguna saat bekerja dengan analisis skala besar (baik dalam ruang maupun waktu) karena membagi data menjadi bagian-bagian yang dapat dikelola dengan mudah dalam memori.
Dask juga dapat menggunakan beberapa inti pemrosesan untuk mempercepat perhitungan.
Hal ini memberikan banyak manfaat bagi analisis, yang akan dibahas dalam notebook ini.




## Deskripsi
Notebook ini membahas cara mengaktifkan Dask sebagai bagian dari proses pemuatan data, yang memungkinkan analisis area yang lebih luas dan rentang waktu yang lebih panjang tanpa menyebabkan lingkungan Piksel mengalami crash, serta berpotensi mempercepat perhitungan.

Topik yang dibahas dalam notebook ini meliputi:

1. Perbedaan antara perintah pemuatan standar dan pemuatan dengan Dask.
2. Mengaktifkan Dask dan Dask Dashboard.
3. Menentukan ukuran chunk untuk pemuatan data.
4. Memuat data dengan Dask.
5. Menggabungkan operasi sebelum memuat data dan memahami grafik tugas (task graphs).

***



## Memulai
Untuk menjalankan pengenalan Dask ini, jalankan semua sel dalam notebook mulai dari sel "Load packages". Untuk bantuan dalam menjalankan sel notebook, lihat kembali notebook [Jupyter notebook](http://43.218.254.133:8888/notebooks/panduan-pengguna/01_Jupyter_notebooks.ipynb) .


### Memuat Paket
Sel di bawah ini mengimpor paket `datacube`, yang sudah menyertakan fungsi Dask.
Paket `dea_tools` menyediakan akses ke fungsi pendukung yang berguna dalam modul `dask`, khususnya fungsi `create_local_dask_cluster`.

In [44]:
import datacube

from dea_tools.dask import create_local_dask_cluster





### Terhubung ke Datacube
Langkah berikutnya adalah menghubungkan ke database datacube.
Objek `dc` yang dihasilkan kemudian dapat digunakan untuk memuat data.
Parameter `app` adalah nama unik yang digunakan untuk mengidentifikasi notebook, tetapi tidak berpengaruh pada analisis.

In [45]:
dc = datacube.Datacube(app="08_parallel_processing_with_dask")



## Pemrosesan Standar
Secara default, pustaka `datacube` **tidak** akan menggunakan Dask saat memuat data.
Artinya, ketika `dc.load()` digunakan, semua data yang terkait dengan kueri pemuatan akan diminta dan dimuat ke dalam memori.

Untuk area yang sangat luas atau rentang waktu yang panjang, hal ini dapat menyebabkan Jupyter Notebook mengalami crash.

Untuk informasi lebih lanjut tentang cara menggunakan `dc.load()`, lihat notebook [Membuka Data](http://43.218.254.133:8888/notebooks/panduan-pengguna/03_Membuka_data.ipynb) dari Piksel.
Di bawah ini, kami menunjukkan contoh pemuatan data standar:

In [47]:


data = dc.load(
    product="s2_l2a",
    measurements=['red', 'green', 'blue'],
    output_crs="EPSG:32748",
    resolution=10,
    time=('2023-01-01', '2023-01-10'),
    longitude=(107.0, 107.1),
    latitude=(-6.6, -6.5),
    dask_chunks={"time": 1, "x": 512, "y": 512},
    group_by="solar_day"
    
)



Querying product Product(name='s2_l2a', id_=9)




## Mengaktifkan Dask
Salah satu fitur utama Dask adalah kemampuannya memanfaatkan beberapa inti CPU untuk mempercepat perhitungan, yang dikenal sebagai komputasi terdistribusi.
Hal ini sangat berguna dalam situasi di mana Anda perlu melakukan banyak perhitungan pada kumpulan data yang besar.

Untuk mengatur komputasi terdistribusi dengan Dask, langkah pertama adalah mengatur klien Dask menggunakan fungsi berikut:

In [48]:
create_local_dask_cluster()

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35995 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/alvioyustian@gmail.com/proxy/35995/status,
Dashboard: /user/alvioyustian@gmail.com/proxy/35995/status,Workers: 1
Total threads: 8,Total memory: 11.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45893,Workers: 0
Dashboard: /user/alvioyustian@gmail.com/proxy/35995/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43931,Total threads: 8
Dashboard: /user/alvioyustian@gmail.com/proxy/37917/status,Memory: 11.40 GiB
Nanny: tcp://127.0.0.1:44745,




Sebuah tampilan output akan muncul, menampilkan informasi tentang `Client` dan `Cluster`.
Untuk saat ini, yang paling penting adalah tautan setelah bagian **Dashboard**: yang terlihat seperti [/user/<email>/proxy/8787/status](#), di mana [\<email\>](#) adalah email Anda untuk Piksel.

Tautan ini memungkinkan Anda untuk melihat bagaimana perhitungan yang sedang dijalankan berkembang. Ada dua cara untuk melihat dasbor ini:

1. Klik tautan tersebut, yang akan membuka tab baru di browser Anda.
2. Mengatur dasbor di dalam lingkungan Piksel.
   
Selanjutnya, kita akan membahas cara melakukan opsi kedua.



### Dashboard Dask di Piksel ###
Pada menu bar di sebelah kiri, klik ikon Dask, seperti yang ditunjukkan di bawah ini:

![Image](DASK_ICON.png)

Salin dan tempel tautan **Dashboard** dari hasil print out Client ke dalam kotak teks DASK DASHBOARD URL:

![Image](DASK_DASHBOARD.png)

Jika URL valid, tombol-tombolnya akan berubah dari abu-abu menjadi oranye.
Klik tombol **PROGRESS** yang berwarna oranye di panel Dask, yang akan membuka tab baru di dalam Lingkungan Piksel.

Untuk melihat jendela Dask dan notebook aktif Anda pada waktu yang bersamaan, seret tab Progress Dask baru ke bagian bawah layar.

Sekarang, ketika Anda melakukan komputasi dengan Dask, Anda akan melihat kemajuan komputasi ini di jendela Dask baru.



## Lazy Load
Saat menggunakan Dask, fungsi `dc.load()` akan beralih dari memuat data secara langsung ke "lazy-loading" data.
Ini berarti data hanya akan dimuat saat diperlukan untuk perhitungan, yang dapat menghemat waktu dan memori.

Lazy-loading mengubah struktur data yang dikembalikan dari perintah `dc.load()`: `xarray.Dataset` yang dikembalikan akan terdiri dari objek `dask.array`.

Untuk meminta data yang dimuat secara tunda, tambahkan parameter `dask_chunks` ke pemanggilan `dc.load()` Anda:

In [57]:
lazy_data = dc.load(
    product="s2_l2a",
    measurements=['red', 'green', 'blue'],
    output_crs="EPSG:32748",
    resolution=10,
    time=('2023-01-01', '2023-01-10'),
    longitude=(107.0, 107.1),
    latitude=(-6.6, -6.5),
    dask_chunks={"time": 1, "x": 3000, "y": 3000},
    group_by="solar_day"
    
)

lazy_data

Querying product Product(name='s2_l2a', id_=9)


<xarray.Dataset> Size: 22MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    green        (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    blue         (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref



Fungsi ini seharusnya mengembalikan hasil jauh lebih cepat, karena tidak ada data yang dibaca dari disk.




### Dask Chunks
Setelah menambahkan parameter `dask_chunks` ke dalam `dc.load()`, data yang di-lazy-loaded akan berisi objek `dask.array` dengan `chunksize` yang terdaftar. `chunksize` ini harus sesuai dengan parameter `dask_chunks` yang diberikan sebelumnya pada pemanggilan `dc.load()`.

Dask bekerja dengan membagi dataset besar menjadi potongan-potongan (chunks), yang dapat dibaca secara individual. Anda dapat menentukan jumlah piksel dalam setiap chunk untuk setiap dimensi dataset.

Sebagai contoh, kami mengirimkan definisi chunk berikut ke dalam `dc.load()`:
```
dask_chunks = {'time': 1, 'x': 3000, 'y': 3000}
```

Definisi ini memberi tahu Dask untuk memotong data menjadi chunk yang berisi 3000 piksel dalam dimensi `x` dan `y`, serta satu ukuran pengukuran dalam dimensi time. Untuk itu, kami selalu mengatur `'time': 1` dalam definisi `dask_chunk`, karena file data hanya mencakup satu waktu pengukuran.

Jika ukuran chunk tidak diberikan untuk dimensi tertentu, atau jika diatur ke -1, maka chunk tersebut akan disesuaikan dengan ukuran array pada dimensi tersebut. Ini berarti semua data pada dimensi tersebut akan dimuat sekaligus, bukannya dibagi menjadi chunk yang lebih kecil.



### Melihat Dask Chunks
Untuk memahami secara visual bagaimana data telah dibagi menjadi chunks, kita dapat menggunakan atribut `.data` yang disediakan oleh `xarray`.

Atribut ini dapat diterapkan pada setiap pengukuran dari data yang di-lazy-loaded. Ketika digunakan dalam Jupyter Notebook, atribut ini akan menampilkan tabel yang merangkum ukuran masing-masing chunk serta jumlah total chunk yang diperlukan.

Contoh di bawah ini menggunakan pengukuran `red` dari data yang di-lazy-loaded:

In [54]:
lazy_data.red.data

dask.array<dc_load_red, shape=(3, 1111, 1112), dtype=uint16, chunksize=(1, 1111, 1112), chunktype=numpy.ndarray>



Dari kolom Chunk pada tabel, kita dapat melihat bahwa data telah dibagi menjadi 4 bagian (chunk), di mana setiap chunk memiliki bentuk `(1 waktu, 3000 piksel, 3000 piksel)` dan membutuhkan memori sebesar 18,00MB.

Jika dibandingkan dengan kolom Array, penggunaan Dask memungkinkan kita untuk memuat 4 bagian data masing-masing sebesar 18,00MB, daripada harus langsung memuat satu bagian besar sebesar 57,67MB.

Pendekatan ini sangat berguna saat bekerja dengan area yang luas atau rentang waktu yang panjang, karena seluruh array mungkin tidak selalu cukup untuk dimuat ke dalam memori yang tersedia. Dengan membagi dataset besar menjadi beberapa chunk dan memuatnya satu per satu, kita dapat melakukan perhitungan pada dataset besar tanpa menyebabkan sistem Piksel mengalami crash.



Saat bekerja dengan data yang di-lazy-loaded, Anda harus secara spesifik meminta Dask untuk membaca dan memuat data ketika ingin menggunakannya.
Sampai Anda melakukan ini, dataset yang di-lazy-loaded hanya mengetahui lokasi data, tetapi tidak mengetahui nilainya.

Untuk memuat data dari disk, gunakan metode `.load()` pada `DataArray` atau `Dataset`.
Jika Anda sudah membuka jendela progres Dask, Anda akan melihat proses komputasi berjalan di sana.

In [55]:
import dask

#Atur scheduler ke "threads" (dijalankan dalam proses yang sama, tidak memerlukan serialisasi)
dask.config.set(scheduler='threads')
loaded_data = lazy_data.load()
loaded_data

<xarray.Dataset> Size: 30MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB 1958 1940 1966 1983 ... 4320 4348 4336
    green        (time, y, x) uint16 7MB 2178 2198 2214 2222 ... 4704 4796 4872
    blue         (time, y, x) uint16 7MB 2054 2034 2048 2058 ... 5196 5220 5148
    nir          (time, y, x) uint16 7MB 4904 4884 4876 4864 ... 5780 5788 5808
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref


Array Dask yang dibuat melalui lazy load
```
red      (time, y, x) uint16 dask.array<chunksize=(1, 3000, 3000), meta=np.ndarray>
```

sekarang telah digantikan dengan angka sebenarnya:

```
 red      (time, y, x) uint16 1958 1940 1966 ... 4320 4348 4336
 ```



Setelah menerapkan perintah `.load()`, data yang di-lazy-loaded menjadi sama dengan data yang dimuat dari query pertama.



## Lazy operations



Selain membagi data menjadi potongan-potongan kecil agar muat di memori, Dask memiliki keunggulan lain, yaitu dapat melacak bagaimana Anda ingin bekerja dengan data dan hanya menjalankan operasi yang diperlukan nanti.

Sekarang, kita akan mengeksplorasi cara kerja ini dengan menghitung Normalized Difference Vegetation Index (NDVI) dari data kita.
Untuk itu, kita akan melakukan lazy load lagi, kali ini dengan menambahkan pita near-infrared (`NIR`) ke dalam perintah `dc.load()`.

In [61]:
lazy_data = dc.load(
    product="s2_l2a",
    measurements=['red', 'green', 'blue','nir'],
    output_crs="EPSG:32748",
    resolution=10,
    time=('2023-01-01', '2023-01-10'),
    longitude=(107.0, 107.1),
    latitude=(-6.6, -6.5),
    dask_chunks={"time": 1, "x": 3000, "y": 3000},
    group_by="solar_day"
    
)

lazy_data

Querying product Product(name='s2_l2a', id_=9)


<xarray.Dataset> Size: 30MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    green        (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    blue         (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    nir          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref



### Menambahkan Lebih Banyak Tugas
Keunggulan utama metode ini adalah kemampuannya untuk merangkai tugas-tugas sebelum memuat data.
Dengan cara ini, Dask hanya akan memuat data yang benar-benar dibutuhkan untuk operasi akhir dalam rantai tugas tersebut.

Kita bisa mendemonstrasikan ini dengan meminta hanya sebagian kecil dari red band.
Jika kita melakukan ini pada data yang dimuat secara lazy, kita bisa melihat grafik tugas (task graph) yang baru.

In [62]:
extract_from_red = lazy_data.red[:, 100:200, 100:200]


Perhatikan bahwa tugas baru `getitem` telah ditambahkan, dan hanya berlaku pada chunk paling kiri.
Jika kita memanggil `.load()` pada array Dask `extract_from_red`, Dask akan melacak operasi kembali melalui grafik tugas untuk menemukan hanya data yang relevan.

Pendekatan ini dapat menghemat memori dan waktu secara signifikan.

Kita bisa memastikan bahwa operasi di atas menghasilkan hasil yang sama seperti memuat data tanpa Dask dan melakukan subset secara manual dengan menjalankan perintah berikut:

In [63]:
lazy_red_subset = extract_from_red.load()
data_red_subset = data.red[:, 100:200, 100:200]

print(f"The loaded arrays match: {lazy_red_subset.equals(data_red_subset)}")

The loaded arrays match: True




Karena array yang dihasilkan sama, maka lebih baik menggunakan lazy-loading untuk merangkai operasi bersama sebelum akhirnya memanggil `.load()`.

Pendekatan ini menghemat waktu dan memori, karena Dask hanya akan memuat data input yang benar-benar diperlukan untuk mendapatkan output akhir.

Dalam contoh ini, lazy-load hanya perlu memuat sebagian kecil dari band red, sedangkan metode pemuatan `data` biasa harus memuat seluruh band `red`, `green`, dan `blue` terlebih dahulu, lalu melakukan subset pada band red.
Akibatnya, waktu dan memori terbuang untuk memuat data yang sebenarnya tidak digunakan.

### Multiple tasks



Keunggulan utama dari lazy-loading dalam Dask adalah kemampuannya untuk merangkai banyak operasi bersama sebelum akhirnya memuat hasil akhir.

Di sini, kita akan merangkai beberapa langkah sekaligus untuk menghitung sebuah band baru dalam array kita, yaitu Normalized Difference Vegetation Index (NDVI).
NDVI dihitung menggunakan band `red` dan `nir`, dengan rumus berikut:

In [64]:
band_diff = lazy_data.nir - lazy_data.red
band_sum = lazy_data.nir + lazy_data.red

lazy_data['ndvi'] = band_diff / band_sum



Dengan melakukan ini, array Dask `ndvi` yang baru ditambahkan ke dalam dataset `lazy_data`.

In [65]:
lazy_data

<xarray.Dataset> Size: 59MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    green        (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    blue         (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    nir          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    ndvi         (time, y, x) float64 30MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref



Terakhir, kita dapat menghitung nilai NDVI dengan memanggil perintah `.load()`.
Kita akan menyimpan hasilnya dalam variabel `ndvi_load`:

In [66]:
ndvi_load = lazy_data.ndvi.load()
ndvi_load

<xarray.DataArray 'ndvi' (time: 3, y: 1111, x: 1112)> Size: 30MB
array([[[0.4293209 , 0.43141852, 0.42531424, ..., 0.37079386,
         0.35006605, 0.36295328],
        [0.43407235, 0.43308007, 0.4237688 , ..., 0.38693787,
         0.36324645, 0.39703034],
        [0.43478261, 0.42907386, 0.41832197, ..., 0.4035313 ,
         0.38734342, 0.40339426],
        ...,
        [0.13182728, 0.13522925, 0.1415903 , ..., 0.40314136,
         0.41602634, 0.42013793],
        [0.12823335, 0.11898734, 0.12863309, ..., 0.39874625,
         0.41290845, 0.41450216],
        [0.13034188, 0.1214697 , 0.12202043, ..., 0.38649308,
         0.40256959, 0.40831557]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.3896178 , 0.39668002, 0.39333523, ..., 0.04463078,
         0.0390941 , 0.0391363 ],
        [0.39289805, 0.3906071 , 0.39428571, ..., 0.04105889,
         0.03946309, 0.03779148],
        [0.39465364, 0.395189  , 0.39465364, ..., 0.03697749,
         0.03933637, 0.03723404],
        ...,
        [0.32354334, 0.26112026, 0.14643603, ..., 0.14451763,
         0.14208494, 0.13026672],
        [0.32657984, 0.29691317, 0.20307048, ..., 0.14487079,
         0.14129159, 0.13385214],
        [0.33169533, 0.29604172, 0.2045538 , ..., 0.14455446,
         0.14206788, 0.14511041]]], shape=(3, 1111, 1112))
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748

Note that running the `.load()` command also modifies the `ndvi` entry in the `lazy_load` dataset:

Perhatikan bahwa menjalankan perintah `.load()` juga memodifikasi entri `ndvi` dalam dataset `lazy_load`:

In [38]:
lazy_data

<xarray.Dataset> Size: 7MB
Dimensions:      (time: 1, y: 667, x: 668)
Coordinates:
  * time         (time) datetime64[ns] 8B 2024-07-12T03:19:56.355000
  * y            (y) float64 5kB 9.288e+06 9.288e+06 ... 9.281e+06 9.281e+06
  * x            (x) float64 5kB 7.277e+05 7.277e+05 ... 7.344e+05 7.344e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 891kB dask.array<chunksize=(1, 100, 100), meta=np.ndarray>
    green        (time, y, x) uint16 891kB dask.array<chunksize=(1, 100, 100), meta=np.ndarray>
    blue         (time, y, x) uint16 891kB dask.array<chunksize=(1, 100, 100), meta=np.ndarray>
    nir          (time, y, x) uint16 891kB dask.array<chunksize=(1, 100, 100), meta=np.ndarray>
    ndvi         (time, y, x) float64 4MB 0.4112 0.2334 0.2028 ... 0.2759 0.557
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


Kamu bisa melihat bahwa `ndvi` adalah sebuah angka, sedangkan semua variabel lainnya merupakan array Dask.



### Menjaga variabel tetap sebagai array Dask
Jika Anda ingin menghitung nilai NDVI tetapi tetap membiarkan `ndvi` sebagai array Dask dalam `lazy_load`, Anda dapat menggunakan perintah `.compute()`.

Untuk mendemonstrasikannya, pertama-tama kita mendefinisikan ulang variabel `ndvi` agar kembali menjadi array Dask.

In [67]:
lazy_data['ndvi'] = band_diff / band_sum
lazy_data

<xarray.Dataset> Size: 59MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    green        (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    blue         (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    nir          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    ndvi         (time, y, x) float64 30MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref


Sekarang, kita melakukan langkah yang sama seperti sebelumnya untuk menghitung NDVI, tetapi menggunakan `.compute()` alih-alih `.load()`:

In [68]:
ndvi_compute = lazy_data.ndvi.compute()
ndvi_compute

<xarray.DataArray 'ndvi' (time: 3, y: 1111, x: 1112)> Size: 30MB
array([[[0.4293209 , 0.43141852, 0.42531424, ..., 0.37079386,
         0.35006605, 0.36295328],
        [0.43407235, 0.43308007, 0.4237688 , ..., 0.38693787,
         0.36324645, 0.39703034],
        [0.43478261, 0.42907386, 0.41832197, ..., 0.4035313 ,
         0.38734342, 0.40339426],
        ...,
        [0.13182728, 0.13522925, 0.1415903 , ..., 0.40314136,
         0.41602634, 0.42013793],
        [0.12823335, 0.11898734, 0.12863309, ..., 0.39874625,
         0.41290845, 0.41450216],
        [0.13034188, 0.1214697 , 0.12202043, ..., 0.38649308,
         0.40256959, 0.40831557]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.3896178 , 0.39668002, 0.39333523, ..., 0.04463078,
         0.0390941 , 0.0391363 ],
        [0.39289805, 0.3906071 , 0.39428571, ..., 0.04105889,
         0.03946309, 0.03779148],
        [0.39465364, 0.395189  , 0.39465364, ..., 0.03697749,
         0.03933637, 0.03723404],
        ...,
        [0.32354334, 0.26112026, 0.14643603, ..., 0.14451763,
         0.14208494, 0.13026672],
        [0.32657984, 0.29691317, 0.20307048, ..., 0.14487079,
         0.14129159, 0.13385214],
        [0.33169533, 0.29604172, 0.2045538 , ..., 0.14455446,
         0.14206788, 0.14511041]]], shape=(3, 1111, 1112))
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748



Anda dapat melihat bahwa nilai telah dihitung, tetapi seperti yang ditunjukkan di bawah ini, variabel `ndvi` tetap sebagai array Dask.

In [69]:
lazy_data

<xarray.Dataset> Size: 59MB
Dimensions:      (time: 3, y: 1111, x: 1112)
Coordinates:
  * time         (time) datetime64[ns] 24B 2023-01-04T03:19:53.464000 ... 202...
  * y            (y) float64 9kB 9.281e+06 9.281e+06 ... 9.27e+06 9.27e+06
  * x            (x) float64 9kB 7.211e+05 7.211e+05 ... 7.322e+05 7.322e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    green        (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    blue         (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    nir          (time, y, x) uint16 7MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
    ndvi         (time, y, x) float64 30MB dask.array<chunksize=(1, 1111, 1112), meta=np.ndarray>
Attributes:
    crs:           EPSG:32748
    grid_mapping:  spatial_ref



Menggunakan `.compute()` memungkinkan Anda menghitung langkah-langkah perantara dan menyimpan hasilnya tanpa mengubah dataset atau array Dask asli. Namun, berhati-hatilah saat menggunakannya, karena dapat menyebabkan kebingungan tentang apa yang telah dan belum dimodifikasi, serta kemungkinan komputasi ulang untuk kuantitas yang sama.

##  Bahan Lanjutan

Untuk bacaan lebih lanjut tentang cara kerja Dask dan bagaimana Dask digunakan oleh xarray, silakan lihat sumber-sumber berikut:

 * http://xarray.pydata.org/en/stable/dask.html
 * https://dask.readthedocs.io/en/latest/
 * http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/

## Langkah-langkah Selanjutnya yang Disarankan

### Untuk notebook ini


Banyak variabel yang digunakan dalam analisis ini dapat dikonfigurasi.
Kami menyarankan untuk kembali ke awal notebook ini dan menjalankan analisis dengan lokasi, tanggal, pengukuran, dan sebagainya yang berbeda.
Ini akan membantu memberikan pemahaman lebih dalam untuk menjalankan analisis Anda sendiri.
Jika Anda belum mencoba aktivitas ekstensi pada percobaan pertama, cobalah dan kerjakan ini saat Anda menjalankan notebook lagi.

### Untuk notebook lainnya


1. [Jupyter Notebooks](http://43.218.254.133:8888/notebooks/panduan-pengguna/01_Jupyter_notebooks.ipynb)
2. [Produk dan Pengukuran](http://43.218.254.133:8888/notebooks/panduan-pengguna/02_Product_dan_measurement.ipynb)
3. [Membuka Data](http://43.218.254.133:8888/notebooks/panduan-pengguna/03_Membuka_data.ipynb)
4. [Memplot Data](http://43.218.254.133:8888/notebooks/panduan-pengguna/04_Plotting.ipynb)
5. [Melakukan Analisis Dasar ]
6. [Pengenalan ke numpy](http://43.218.254.133:8888/notebooks/panduan-pengguna/06_numpy.ipynb)
7. [Pengenalan ke xarray](http://43.218.254.133:8888/notebooks/panduan-pengguna/07_xarray.ipynb)
8. **Pemrosesan Paralel dengan Dask (notebook ini)**


Setelah Anda menyelesaikan tutorial di atas, bergabunglah dengan pengguna tingkat lanjut untuk menjelajahi:

Direktori "Datasets" dalam repositori, tempat Anda dapat menjelajahi produk DE Africa secara mendalam.
Direktori "Frequently used code", yang berisi buku resep teknik dan metode umum untuk menganalisis data DE Africa.
Direktori "Real-world examples", yang menyediakan alur kerja dan studi kasus analisis yang lebih kompleks.



## Informasi Tambahan

**Lisensi:** Kode dalam notebook ini dilisensikan di bawah [Lisensi Apache, Versi 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Kontak:** Jika Anda memerlukan bantuan, silakan ajukan pertanyaan di [Saluran Slack Open Data Cube](http://slack.opendatacube.org/) atau di [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) menggunakan tag `open-data-cube` (Anda dapat melihat pertanyaan yang sebelumnya diajukan [di sini](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
Jika Anda ingin melaporkan masalah dengan notebook ini, Anda dapat mengajukannya di [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Versi datacube yang kompatibel:**

In [70]:
print(datacube.__version__)

1.9.6


**Last Tested:** 

In [71]:
from datetime import date
print(date.today())

2025-07-31
